# Using YAHPO Gym: A quick introduction

Using YAHPO GYM we can benchmark a new Hyperparameter optimization method on a large amount of problems in a very short time-frame.

This tutorial walks us through the core concepts and functionality of ``yahpo_gym` and showscases a practical example.

YAHPO GYM consists of several `scenarios`, e.g. the collection of all benchmark instances in `lcbench` is a `scenario`.
An `instance` is the concrete task of optimizing hyperparameters of the neural network on a given dataset from OpenML.


## Core functionality: Configuration & BenchmarkSet

We first a have a brief look at at the two core classes we will make use of in `YAHPO GYM`: 
- A `Configuration` contains all relevant infos regarding a specific benchmarking scenario e.g. `lcbench`. We can load configurations with the `cfg(<key>)` shortcut.
- A `BenchmarkSet` can be instantiated using a Configuration (or it's key) and contains the logic used to evaluate the surrogate model for a given query hyperparameter configuration (or set thereof).b


To run a benchmark you need to download the ONNX model (`new_model.onnx`), [ConfigSpace](https://automl.github.io/ConfigSpace/) (`config_space.json`) and some encoding info (`encoding.json`).

You can download these [here](https://syncandshare.lrz.de/getlink/fiCMkzqj1bv1LfCUyvZKmLvd/), but **YAHPO GYM** can also autmatically download this for you.

You should pertain the folder structure as on the hosting site (i.e., create a `"path-to-data"` directory, for example named `"multifidelity_data"`, containing the individual, e.g., `"lcench"`, directories).

In [7]:
# Initialize the local config & set path for surrogates and metadata
from yahpo_gym import local_config
local_config.init_config()
local_config.set_data_path("~/yahpo_models")

### Configuration

In [8]:
# We first load the dict of configurations and the concrete benchmarks
%load_ext autoreload
%autoreload 2
from yahpo_gym.configuration import cfg
import yahpo_gym.benchmarks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Now we can print a list of available configurations:



In [10]:
# And instantiate a Configuration using a key.
conf_lcb = cfg('lcbench')

In [11]:
# We can download all required files using `download_files`.
# Files will be downloaded to the data_path ("~/yahpo_models") set above.
conf_lcb.download_files()

This allows us to query several important properties of the benchmark problem:

- config_id : The id / key of the configuration
- y_names  : The names of the target variables included in the surrogate model
- hp_names: The names of all hyperparameters
- cat_names : The names of categorical hyperparameters
- cont_names  :  The names of continuous hyperparameters
- fidelity_params  : The name of the fidelity parameter(s)
- instance_names : The column pertaining to the available instances in a dataset
- runtime_name : The name of parameters remeasuring runtime of  the model. 
- data : A `pandas` `DataFrame` containing the data used to train the surrogates. Only available if the data was downloaded.

In [12]:
# We can for example query the target outputs of our surrogate:
conf_lcb.y_names

['time',
 'val_accuracy',
 'val_cross_entropy',
 'val_balanced_accuracy',
 'test_cross_entropy',
 'test_balanced_accuracy']

### BemchmarkSet

A benchmark set allows us to evaluate the surrogate models for a given configuration.
We can instantiate them similarly to a `Configuration` using the **key**.

In [13]:
from yahpo_gym import benchmark_set
# Select a Benchmark
bench = benchmark_set.BenchmarkSet("lcbench")
bench

BenchmarkInstance (lcbench)

This can again be used to query relevant meta-information:
- instances: The available instances (in this case OpenML Task Id's)

In [14]:
cfg("lcbench").config

{'basedir': Path('/home/lps/yahpo_models'),
 'download_url': 'https://syncandshare.lrz.de/dl/fiCMkzqj1bv1LfCUyvZKmLvd',
 'config_id': 'lcbench',
 'model': 'new_model.onnx',
 'dataset': 'data.csv',
 'config_space': 'config_space.json',
 'encoding': 'encoding.json',
 'y_names': ['time',
  'val_accuracy',
  'val_cross_entropy',
  'val_balanced_accuracy',
  'test_cross_entropy',
  'test_balanced_accuracy'],
 'cont_names': ['epoch',
  'batch_size',
  'learning_rate',
  'momentum',
  'weight_decay',
  'num_layers',
  'max_units',
  'max_dropout'],
 'cat_names': ['OpenML_task_id'],
 'fidelity_params': ['epoch'],
 'runtime_name': 'time',
 'model_old': 'model.onnx',
 'y_minimize': [True, False, True, False, True, False],
 'instance_names': 'OpenML_task_id'}

In [15]:
# List available instances
bench.instances[0:5]

['3945', '7593', '34539', '126025', '126026']

We can now set an instance, this defines the instance (i.e. concrete dataset) to be evaluated.
We can furthermore use the included `ConfigSpace` in order to sample a concrete configuration and evaluate it: 

In [16]:
# Set an instance
bench.set_instance("3945")
# Sample a point from the configspace (containing parameters for the instance and budget)
value = bench.config_space.sample_configuration(1).get_dictionary()
# Evaluate
print(bench.objective_function(value))

{'time': 1.0176891, 'val_accuracy': 96.08849, 'val_cross_entropy': 0.36274177, 'val_balanced_accuracy': 0.533724, 'test_cross_entropy': 0.5219554, 'test_balanced_accuracy': 0.53069955}


In [17]:
value

{'OpenML_task_id': '3945',
 'batch_size': 33,
 'epoch': 29,
 'learning_rate': 0.004109761904272161,
 'max_dropout': 0.23517516037170827,
 'max_units': 65.27423374909003,
 'momentum': 0.27385920624918386,
 'num_layers': 4,
 'weight_decay': 0.06160935519075886}

In [18]:
# And the corresponding space we optimize over.
bench.get_opt_space()

Configuration space object:
  Hyperparameters:
    OpenML_task_id, Type: Constant, Value: 3945
    batch_size, Type: UniformInteger, Range: [16, 512], Default: 91, on log-scale
    learning_rate, Type: UniformFloat, Range: [0.00010000000000000009, 0.10000000000000002], Default: 0.0031622777, on log-scale
    max_dropout, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5
    max_units, Type: UniformFloat, Range: [63.99999999999998, 1024.0], Default: 256.0, on log-scale
    momentum, Type: UniformFloat, Range: [0.1, 0.99], Default: 0.545
    num_layers, Type: UniformInteger, Range: [1, 5], Default: 3
    weight_decay, Type: UniformFloat, Range: [1e-05, 0.1], Default: 0.050005

## A working example

In order to demonstrate using YAHPO Gym more in-depth we provide a full example benchmarking `HPBandSter` on an `lcbench` task.
We again start by importing the relevant modules:

In [19]:
from yahpo_gym import benchmark_set
import yahpo_gym.benchmarks.lcbench
import time
import numpy as np

Now we can define a worker class as required by `HPBandSter` that internally calls our `objective_function`.

In [20]:
from hpbandster.core.worker import Worker
import hpbandster.core.nameserver as hpns
from hpbandster.optimizers import BOHB as BOHB

class lcbench(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.bench = bench
        self.sleep_interval = sleep_interval

    def compute(self, config, budget, **kwargs):
        """
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of epochs the model can use to train

        Returns:
            dictionary with mandatory fields:
                "loss" (scalar)
                "info" (dict)
        """

        config.update({"epoch": int(np.round(budget))})  # update epoch
        result = bench.objective_function(config)  # evaluate

        time.sleep(self.sleep_interval)

        return({
                    "loss": - float(result.get("val_accuracy")),  # we want to maximize validation accuracy
                    "info": "empty"
                })
    
    @staticmethod
    def get_configspace():
        # sets OpenML_task_id constant to "3945" and removes the epoch fidelity parameter
        cs = bench.get_opt_space(instance = "3945", drop_fidelity_params = True)
        return(cs)

Using this worker class, we can now run the full benchmark:

In [21]:
# Initialize the set providing a scenario
bench = benchmark_set.BenchmarkSet("lcbench")
# Choose an instance
bench.set_instance("3945")

# Start up the nameserver
NS = hpns.NameServer(run_id="lcbench", host="127.0.0.1", port=None)
NS.start()

# Run BOHB
w = lcbench(sleep_interval=0, nameserver="127.0.0.1", run_id ="lcbench")
w.run(background=True)

bohb = BOHB(configspace=w.get_configspace(),
            run_id="lcbench", nameserver="127.0.0.1",
            min_budget=1, max_budget=52)

res = bohb.run(n_iterations=1)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

18:34:58 wait_for_workers trying to get the condition
18:34:58 DISPATCHER: started the 'discover_worker' thread
18:34:58 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7f906d6d6908; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
18:34:58 DISPATCHER: started the 'job_runner' thread
18:34:58 WORKER: No dispatcher found. Waiting for one to initiate contact.
18:34:58 WORKER: start listening for jobs
18:34:58 DISPATCHER: Pyro daemon running on localhost:38707
18:34:58 DISPATCHER: Starting worker discovery
18:34:58 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
18:34:58 DISPATCHER: discovered new worker, hpbandster.run_lcbench.worker.arch.56954140259706070848
18:34:58 HBMASTER: number of workers changed to 1
18:34:58 Enough workers to start this run!
18:34:58 HBMASTER: starting run at 1632933298.880362
18:34:58 adjust_queue_size: lock accquired
18:34:58 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:34:58 HBMASTER: adjusted queue 

18:34:59 Only 3 run(s) for budget 1.925926 available, need more than 10 -> can't build model!
18:34:59 HBMASTER: Trying to run another job!
18:34:59 job_callback for (0, 0, 2) finished
18:34:59 start sampling a new configuration.
18:34:59 done sampling a new configuration.
18:34:59 HBMASTER: schedule new run for iteration 0
18:34:59 HBMASTER: trying submitting job (0, 0, 3) to dispatcher
18:34:59 HBMASTER: submitting job (0, 0, 3) to dispatcher
18:34:59 DISPATCHER: trying to submit job (0, 0, 3)
18:34:59 DISPATCHER: trying to notify the job_runner thread.
18:34:59 HBMASTER: job (0, 0, 3) submitted to dispatcher
18:34:59 DISPATCHER: Trying to submit another job.
18:34:59 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
18:34:59 DISPATCHER: starting job (0, 0, 3) on hpbandster.run_lcbench.worker.arch.56954140259706070848
18:34:59 DISPATCHER: job (0, 0, 3) dispatched on hpbandster.run_lcbench.worker.arch.56954140259706070848
18:34:59 WORKER: start processing job (0, 0, 3)
18:34:59 D

18:34:59 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
18:34:59 WORKER: args: ()
18:34:59 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 159, 'learning_rate': 0.0007794238351845806, 'max_dropout': 0.6177280454429146, 'max_units': 284.36286806012544, 'momentum': 0.5467757478844134, 'num_layers': 4, 'weight_decay': 0.015343310867126706}, 'budget': 1.9259259259259258, 'working_directory': '.'}
18:34:59 WORKER: done with job (0, 0, 6), trying to register it.
18:34:59 WORKER: registered result for job (0, 0, 6) with dispatcher
18:34:59 DISPATCHER: job (0, 0, 6) finished
18:34:59 DISPATCHER: register_result: lock acquired
18:34:59 DISPATCHER: job (0, 0, 6) on hpbandster.run_lcbench.worker.arch.56954140259706070848 finished
18:34:59 job_id: (0, 0, 6)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 159, 'learning_rate': 0.0007794238351845806, 'max_dropout': 0.6177280454429146, 'max_units': 284.36286806012544, 'momentum': 0.5467757478844

18:34:59 job_callback for (0, 0, 9) started
18:34:59 DISPATCHER: Trying to submit another job.
18:34:59 job_callback for (0, 0, 9) got condition
18:34:59 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:34:59 HBMASTER: Trying to run another job!
18:34:59 job_callback for (0, 0, 9) finished
18:34:59 start sampling a new configuration.
18:34:59 done sampling a new configuration.
18:34:59 HBMASTER: schedule new run for iteration 0
18:34:59 HBMASTER: trying submitting job (0, 0, 10) to dispatcher
18:34:59 HBMASTER: submitting job (0, 0, 10) to dispatcher
18:34:59 DISPATCHER: trying to submit job (0, 0, 10)
18:34:59 DISPATCHER: trying to notify the job_runner thread.
18:34:59 HBMASTER: job (0, 0, 10) submitted to dispatcher
18:34:59 DISPATCHER: Trying to submit another job.
18:34:59 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
18:34:59 DISPATCHER: starting job (0, 0, 10) on hpbandster.run_lcbench.worker.arch.56954140259706070848
18:34:59 DISPATCHER: job (0

18:34:59 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 86, 'learning_rate': 0.006589518223952934, 'max_dropout': 0.8451177276334075, 'max_units': 205.2164637516088, 'momentum': 0.44951073668224517, 'num_layers': 3, 'weight_decay': 0.028528629600967963}, 'budget': 1.9259259259259258, 'working_directory': '.'}
18:34:59 WORKER: done with job (0, 0, 13), trying to register it.
18:34:59 WORKER: registered result for job (0, 0, 13) with dispatcher
18:34:59 DISPATCHER: job (0, 0, 13) finished
18:34:59 DISPATCHER: register_result: lock acquired
18:34:59 DISPATCHER: job (0, 0, 13) on hpbandster.run_lcbench.worker.arch.56954140259706070848 finished
18:34:59 job_id: (0, 0, 13)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 86, 'learning_rate': 0.006589518223952934, 'max_dropout': 0.8451177276334075, 'max_units': 205.2164637516088, 'momentum': 0.44951073668224517, 'num_layers': 3, 'weight_decay': 0.028528629600967963}, 'budget': 1.9259259259259258, 'working_direc

...

18:35:01 WORKER: done with job (0, 0, 23), trying to register it.
18:35:01 WORKER: registered result for job (0, 0, 23) with dispatcher
18:35:01 DISPATCHER: job (0, 0, 23) finished
18:35:01 DISPATCHER: register_result: lock acquired
18:35:01 DISPATCHER: job (0, 0, 23) on hpbandster.run_lcbench.worker.arch.56954140259706070848 finished
18:35:01 job_id: (0, 0, 23)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 21, 'learning_rate': 0.018551367122870657, 'max_dropout': 0.2997234441734993, 'max_units': 165.58415856603133, 'momentum': 0.4868101428201148, 'num_layers': 3, 'weight_decay': 0.009152961918547352}, 'budget': 1.9259259259259258, 'working_directory': '.'}
result: {'loss': -92.36121368408203, 'info': 'empty'}
exception: None

18:35:01 job_callback for (0, 0, 23) started
18:35:01 DISPATCHER: Trying to submit another job.
18:35:01 job_callback for (0, 0, 23) got condition
18:35:01 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:35:01 done building

18:35:01 DISPATCHER: job (0, 0, 26) finished
18:35:01 DISPATCHER: register_result: lock acquired
18:35:01 DISPATCHER: job (0, 0, 26) on hpbandster.run_lcbench.worker.arch.56954140259706070848 finished
18:35:01 job_id: (0, 0, 26)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 28, 'learning_rate': 0.006460090313191405, 'max_dropout': 0.15578035756628578, 'max_units': 83.44448456011497, 'momentum': 0.49302705084935683, 'num_layers': 4, 'weight_decay': 0.07562072751085074}, 'budget': 1.9259259259259258, 'working_directory': '.'}
result: {'loss': -88.32975006103516, 'info': 'empty'}
exception: None

18:35:01 job_callback for (0, 0, 26) started
18:35:01 DISPATCHER: Trying to submit another job.
18:35:01 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:35:01 job_callback for (0, 0, 26) got condition
18:35:01 done building a new model for budget 1.925926 based on 9/22 split
Best loss for this budget:-92.361214





18:35:01 HBMASTER: Trying to run another 

18:35:01 job_callback for (0, 0, 15) started
18:35:01 DISPATCHER: Trying to submit another job.
18:35:01 job_callback for (0, 0, 15) got condition
18:35:01 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:35:01 Only 3 run(s) for budget 5.777778 available, need more than 10 -> can't build model!
18:35:01 HBMASTER: Trying to run another job!
18:35:01 job_callback for (0, 0, 15) finished
18:35:01 HBMASTER: schedule new run for iteration 0
18:35:01 HBMASTER: trying submitting job (0, 0, 19) to dispatcher
18:35:01 HBMASTER: submitting job (0, 0, 19) to dispatcher
18:35:01 DISPATCHER: trying to submit job (0, 0, 19)
18:35:01 DISPATCHER: trying to notify the job_runner thread.
18:35:01 HBMASTER: job (0, 0, 19) submitted to dispatcher
18:35:01 DISPATCHER: Trying to submit another job.
18:35:01 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
18:35:01 DISPATCHER: starting job (0, 0, 19) on hpbandster.run_lcbench.worker.arch.56954140259706070848
18:35:01 DISPATCHER

18:35:02 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 29, 'learning_rate': 0.014559531021697892, 'max_dropout': 0.015117747090709566, 'max_units': 135.66679496038773, 'momentum': 0.5471314742222566, 'num_layers': 3, 'weight_decay': 0.02185371141925171}, 'budget': 5.777777777777778, 'working_directory': '.'}
18:35:02 WORKER: done with job (0, 0, 24), trying to register it.
18:35:02 WORKER: registered result for job (0, 0, 24) with dispatcher
18:35:02 DISPATCHER: job (0, 0, 24) finished
18:35:02 DISPATCHER: register_result: lock acquired
18:35:02 DISPATCHER: job (0, 0, 24) on hpbandster.run_lcbench.worker.arch.56954140259706070848 finished
18:35:02 job_id: (0, 0, 24)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 29, 'learning_rate': 0.014559531021697892, 'max_dropout': 0.015117747090709566, 'max_units': 135.66679496038773, 'momentum': 0.5471314742222566, 'num_layers': 3, 'weight_decay': 0.02185371141925171}, 'budget': 5.777777777777778, 'working_direc

18:35:02 DISPATCHER: Trying to submit another job.
18:35:02 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
18:35:02 job_callback for (0, 0, 19) got condition
18:35:02 Only 1 run(s) for budget 17.333333 available, need more than 10 -> can't build model!
18:35:02 HBMASTER: Trying to run another job!
18:35:02 job_callback for (0, 0, 19) finished
18:35:02 HBMASTER: schedule new run for iteration 0
18:35:02 HBMASTER: trying submitting job (0, 0, 23) to dispatcher
18:35:02 HBMASTER: submitting job (0, 0, 23) to dispatcher
18:35:02 DISPATCHER: trying to submit job (0, 0, 23)
18:35:02 DISPATCHER: trying to notify the job_runner thread.
18:35:02 HBMASTER: job (0, 0, 23) submitted to dispatcher
18:35:02 DISPATCHER: Trying to submit another job.
18:35:02 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
18:35:02 DISPATCHER: starting job (0, 0, 23) on hpbandster.run_lcbench.worker.arch.56954140259706070848
18:35:02 DISPATCHER: job (0, 0, 23) dispatched on hpbandster.ru

and print the results

In [22]:
id2config = res.get_id2config_mapping()
id2config

{(0,
  0,
  0): {'config': {'OpenML_task_id': '3945',
   'batch_size': 74,
   'learning_rate': 0.03730364728284649,
   'max_dropout': 0.10928899279517446,
   'max_units': 352.7361539760635,
   'momentum': 0.6132073024377236,
   'num_layers': 1,
   'weight_decay': 0.04487797472390004}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'OpenML_task_id': '3945',
   'batch_size': 31,
   'learning_rate': 0.08518909658125563,
   'max_dropout': 0.3058476368770261,
   'max_units': 863.0596531269018,
   'momentum': 0.5368413097989411,
   'num_layers': 4,
   'weight_decay': 0.08674342468960657}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'OpenML_task_id': '3945',
   'batch_size': 29,
   'learning_rate': 0.00013186846924119304,
   'max_dropout': 0.5978682680734122,
   'max_units': 408.45051767218047,
   'momentum': 0.2511205608534902,
   'num_layers': 2,
   'weight_decay': 0.00969841062711537}, 'config_info': {'model_based_pick': False}},
 (0,


In [23]:
incumbent = res.get_incumbent_id()
incumbent

(0, 0, 23)

In [24]:

print("Best found configuration:", id2config[incumbent]["config"])
print("A total of %i unique configurations where sampled." % len(id2config.keys()))
print("A total of %i runs where executed." % len(res.get_all_runs()))
print("Total budget corresponds to %.1f full function evaluations."%(sum([r.budget for r in res.get_all_runs()])/1))

Best found configuration: {'OpenML_task_id': '3945', 'batch_size': 21, 'learning_rate': 0.018551367122870657, 'max_dropout': 0.2997234441734993, 'max_units': 165.58415856603133, 'momentum': 0.4868101428201148, 'num_layers': 3, 'weight_decay': 0.009152961918547352}
A total of 27 unique configurations where sampled.
A total of 40 runs where executed.
Total budget corresponds to 208.0 full function evaluations.
